In [1]:
from typing import *
import collections
import heapq
import math
import bisect
import functools

# 6233. Convert the Temperature

In [2]:
class Solution:
    def convertTemperature(self, celsius: float) -> List[float]:
        return [celsius + 273.15, celsius * 1.8 + 32]

# 6234. Number of Subarrays With LCM Equal to K

In [4]:
class Solution:
    def subarrayLCM(self, nums: List[int], k: int) -> int:
        ret = 0
        for i, num in enumerate(nums):
            sums = num
            if sums == k:
                ret += 1
            for j in range(i+1, len(nums)):
                sums = sums * nums[j] // math.gcd(sums, nums[j])
                if sums == k:
                    ret += 1
        return ret

# 6235. Minimum Number of Operations to Sort a Binary Tree by Level

In [5]:
# Definition for a binary tree node.
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

class Union:
    def __init__(self, lens):
        self.Union = {}
        self.rank = {}
        self.nums = lens-1

    def find(self, x):
        self.Union.setdefault(x, x)
        while self.Union[x] != x:
            x = self.Union[x]
        return x

    def union(self, x, y):
        par_x, par_y = self.find(x), self.find(y)
        if par_x == par_y:
            return
        self.rank.setdefault(par_x, 1)
        self.rank.setdefault(par_y, 1)
        if self.rank[par_x] < self.rank[par_y]:
            par_x, par_y = par_y, par_x
        self.rank[par_x] += self.rank[par_y]
        self.Union[par_y] = par_x
        self.nums -= 1

    def is_union(self, x, y):
        par_x, par_y = self.find(x), self.find(y)
        if par_x == par_y:
            return True
        return False
        
class Solution:
    def minimumOperations(self, root: Optional[TreeNode]) -> int:
        q = collections.deque([root])
        ret = 0
        while q:
            newq = collections.deque()
            while q:
                node = q.popleft()
                if node.left:
                    newq.append(node.left)
                if node.right:
                    newq.append(node.right)
            
            union = Union(len(newq))
            sortedq = sorted(newq, key=lambda x:x.val)
            for i in range(len(sortedq)):
                if sortedq[i].val != newq[i].val:
                    for j, n in enumerate(newq):
                        if n.val == sortedq[i].val:
                            idx = j
                            break
                    union.union(i, idx)
            ret += sum(union.rank) - len(union.rank)
        return ret

In [71]:
class Solution:
    def minimumOperations(self, root: Optional[TreeNode]) -> int:
        ans, q = 0, [root]
        while q:
            a = []
            tmp = q
            q = []
            for node in tmp:
                a.append(node.val)
                if node.left: q.append(node.left)
                if node.right: q.append(node.right)

            b = sorted(a)
            a = [bisect_left(b, v) for v in a]  # 离散化

            ans += len(a)
            vis = [False] * len(a)
            for v in a:
                if vis[v]: continue
                while not vis[v]:
                    vis[v] = True
                    v = a[v]
                ans -= 1
        return ans

# 6236. Maximum Number of Non-overlapping Palindrome Substrings

In [67]:
class Solution:
    def maxPalindromes(self, s: str, k: int) -> int:
        a = [[0]*len(s) for _ in range(len(s))]
        for i, n in enumerate(s):
            a[i][i] = 1
            if i < len(s)-1 and n == s[i+1]:
                a[i][i+1] = 1
        
        for i in range(len(s)-1, -1, -1):
            for j in range(i+2, len(s)):
                if s[i] == s[j] and a[i+1][j-1]:
                    a[i][j] = 1
        
        dp = [0] * (len(s)+1)
        for i in range(len(s)):
            flag = 0
            for j in range(i-k+1, -1, -1):
                flag = flag | a[j][i]
                dp[i+1] = max(dp[i+1], dp[j] + flag)
        return max(dp)

In [97]:
s = "abaccdbbd"
k = 3

In [82]:
s = "kwnwkekokedadq"
k = 5

In [83]:
a = Solution().maxPalindromes(s, k)
a

[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2]


2

In [81]:
class Solution:
    def maxPalindromes(self, s: str, k: int) -> int:
        lens = len(s)
        dp = [0] * (lens+1)
        for i in range(lens*2-1):
            l, r = i //2, i//2 + (i%2)
            dp[r+1] = max(dp[l+1], dp[r+1])
            while r < lens and l >=0 and s[l] == s[r]:
                if r-l+1 >= k:
                    dp[r+1] = max(dp[r+1], dp[l]+1)
                l -= 1
                r += 1
        return dp[-1]

In [85]:
class Manacher:
    @staticmethod
    def trans(s):
        ret = ["#"] * (len(s)*2+1)
        for i, n in enumerate(s):
            ret[2*i+1] = n
        return ret

    def manacher(self, s):
        new_s = self.trans(s)
        lens = len(new_s)
        c, r = -1, -1
        i = 0
        re = [0] * lens
        ret = 1
        while i < lens:
            # original idea
            # if i > c+r:
            #     i_r = 1
            #     while i-i_r >= 0 and i+i_r < lens and new_s[i-i_r] == new_s[i+i_r]:
            #         i_r += 1
            #     re[i] = i_r-1
            #     c, r = i, i_r-1
            # else:
            #     ti = c - (i - c)
            #     if ti-re[ti] > c-r:
            #         re[i] = re[ti]
            #     elif ti-re[ti] < c-r:
            #         re[i] = ti - (c-r)
            #     else:
            #         i_r = ti - (c-r)+1
            #         while i - i_r >= 0 and i + i_r < lens and new_s[i - i_r] == new_s[i + i_r]:
            #             i_r += 1
            #         re[i] = i_r-1
            #         c, r = i, i_r-1
            # i += 1

            # optimization
            i_r = 1 if i > c+r else min(re[2*c-i], c-i+r) + 1
            while i - i_r >= 0 and i + i_r < lens and new_s[i - i_r] == new_s[i + i_r]:
                i_r += 1
            i_r -= 1
            re[i] = i_r
            ret = max(ret, i_r)
            if i_r + i > c + r:
                c, r = i, i_r
            i += 1
        return ret, re

In [89]:
s

'kwnwkekokedadq'

In [90]:
ret, re = Manacher().manacher("ss")
print(re)

[0, 1, 2, 1, 0]


In [120]:
class Solution:
    def maxPalindromes(self, s: str, k: int) -> int:
        re = Manacher().manacher(s)[1][1:]
        lens = len(s)
        dp = [0] * (lens+1)
        for i, n in enumerate(s):
            dp[i+1] = max(dp[i], dp[i+1])
            if re[i*2] >= k:
                dp[i+k//2+1] = max(dp[i+k//2+1], dp[i-k//2]+1)
            if (k%2 and re[i*2+1] >= k+1) or (k%2==0 and re[i*2+1] >= k):
                dp[i+(k+1)//2+1] = max(dp[i+(k+1)//2+1], dp[i-(k+1)//2+1]+1)
        return dp[-1]

In [121]:
a = Solution().maxPalindromes(s, k)
a

5

In [115]:
s =  "qvlbphaahpblshrwzzwrhscaivaeffllffeanwtlaaltwngtupwkwwkwpuaa"
k = 10